It's taken way too long to get to weather, this will only be quite a superficial look into that.

In [11]:
import pandas as pd
import pgaccess as pg

In [89]:
weather = pd.read_csv('../data/delta_weather.csv', parse_dates=['Date time'])

carrierDelay = pd.read_csv('../data/monthly_carrier_delay.csv')
carrierDelay.set_index(['op_unique_carrier', 'month'], inplace=True)

planeSpeed = pd.read_csv('../data/plane_speed.csv')
planeSpeed.set_index(['tail_num'], inplace=True)

ratios = pd.read_csv('../data/airport_delays.csv', index_col=0)
dists = pd.read_csv('../data/distance_delays.csv')
months = pd.read_csv('../data/month_delays.csv')
hours = pd.read_csv('../data/hour_delays.csv')

In [23]:
cities = [
    "Chicago, IL", "Atlanta, GA", "New York, NY", "Dallas/Fort Worth, TX",
    "Denver, CO", "Charlotte, NC", "Los Angeles, CA", "Washington, DC",
    "Houston, TX", "Seattle, WA"
]
sqlfilter = f"""'{"', '".join(cities)}'"""
flightData = pg.get_filtered_test_data(500000, f'''
    (origin_city_name IN ({sqlfilter}) OR
    dest_city_name IN ({sqlfilter}))
''')
flightData

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,target
0,2018-04-18,DL,DL,DL,1781,DL,N946DN,1781,10397,ATL,...,13303,MIA,"Miami, FL",1150,1342,N,112.0,1.0,594.0,-17.0
1,2018-04-18,DL,DL,DL,1790,DL,N912DN,1790,14683,SAT,...,10397,ATL,"Atlanta, GA",1400,1723,N,143.0,1.0,874.0,-19.0
2,2018-04-18,DL,DL,DL,1813,DL,N961AT,1813,10397,ATL,...,15249,TLH,"Tallahassee, FL",1623,1724,N,61.0,1.0,223.0,-10.0
3,2018-04-18,DL,DL,DL,1841,DL,N692DL,1841,12892,LAX,...,13487,MSP,"Minneapolis, MN",1220,1803,N,223.0,1.0,1535.0,4.0
4,2018-04-18,DL,DL,DL,1911,DL,N995AT,1911,10397,ATL,...,11953,GNV,"Gainesville, FL",1954,2110,N,76.0,1.0,300.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316605,2018-04-18,DL,DL,DL,1617,DL,N906DL,1617,14027,PBI,...,10397,ATL,"Atlanta, GA",545,735,N,110.0,1.0,545.0,-5.0
316606,2018-04-18,DL,DL,DL,1618,DL,N369NB,1618,12892,LAX,...,13198,MCI,"Kansas City, MO",900,1420,N,200.0,1.0,1363.0,-14.0
316607,2018-04-18,DL,DL,DL,1660,DL,N3740C,1660,11433,DTW,...,14747,SEA,"Seattle, WA",1750,1933,N,283.0,1.0,1927.0,-1.0
316608,2018-04-18,DL,DL,DL,1672,DL,N918DL,1672,10792,BUF,...,10397,ATL,"Atlanta, GA",1059,1304,N,125.0,1.0,712.0,-9.0


This block is ported basically directly from Predict Delay 2

Just done as a function to keep it contained. I may move it to another .py file at some point if I actually use it more than once.

In [88]:
def predict2(flightData, carrierDelay, planeSpeed, ratios, dists, months, hours):
    modelData = flightData.copy()
    modelData['month'] = modelData.fl_date.dt.month

    # Monthly carrier delay
    modelData['carrier_delay'] = pd.merge(
        modelData, carrierDelay,
        left_on = ['op_unique_carrier', 'month'],
        right_index = True
    )['mean_delay']

    # Plane speed
    modelData['plane_speed'] = pd.merge(
        modelData, planeSpeed,
        left_on = 'tail_num',
        right_index = True
    )['speed']
    
    modelData = pd.merge(
        pd.merge(modelData, ratios[['origin_ratio']], left_on='origin_airport_id', right_index=True),
        ratios[['dest_ratio']],
        left_on = 'dest_airport_id',
        right_index = True
    )
    modelData = pd.merge(
        pd.merge(modelData, dists),
        months
    )
    
    modelData['hour'] = modelData['crs_arr_time'] // 100
    modelData = pd.merge(modelData, hours)
    
    return modelData

modelData = predict2(flightData, carrierDelay, planeSpeed, ratios, dists, months, hours)
modelData

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,target,month,carrier_delay,plane_speed,origin_ratio,dest_ratio,distance_ratio,month_ratio,hour,hour_ratio
0,2018-04-18,DL,DL,DL,1781,DL,N946DN,1781,10397,ATL,...,-17.0,4,0.163906,0.147540,5.761939e+05,176004.521277,23328.456293,4.474688e+07,13,8921628.0
1,2019-04-16,AA,AA,AA,2828,AA,N991AN,2828,10397,ATL,...,9.0,4,4.239740,0.143477,5.761939e+05,176004.521277,23328.456293,4.474688e+07,13,8921628.0
2,2018-04-26,DL,DL,DL,1987,DL,N934DL,1987,13303,MIA,...,-20.0,4,0.163906,0.158601,3.112660e+05,413880.585106,23328.456293,4.474688e+07,13,8921628.0
3,2019-04-12,DL,DL,DL,1210,DL,N914DN,1210,13303,MIA,...,-4.0,4,0.163906,0.148119,3.112660e+05,413880.585106,23328.456293,4.474688e+07,13,8921628.0
4,2019-04-11,AA,AA,AA,2669,AA,N842NN,2669,13303,MIA,...,-10.0,4,4.239740,0.143921,3.112660e+05,968278.457447,4990.204588,4.474688e+07,13,8921628.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316605,2019-01-14,B6,B6,B6,499,B6,N562JB,499,12953,LGA,...,55.0,1,14.760556,0.137919,5.650231e+05,465330.319149,8279.603224,4.016998e+07,24,16216.0
316606,2019-11-06,NK,NK,NK,968,NK,N687NK,968,13930,ORD,...,2.0,11,1.105945,0.136459,2.238224e+06,130999.734043,6480.099194,3.189132e+07,24,16216.0
316607,2019-02-04,B6,B6,B6,499,B6,N583JB,499,12953,LGA,...,-22.0,2,9.964439,0.140308,5.650231e+05,465330.319149,8279.603224,6.279128e+07,24,16216.0
316608,2019-02-17,B6,B6,B6,499,B6,N640JB,499,12953,LGA,...,59.0,2,9.964439,0.137931,5.650231e+05,465330.319149,8279.603224,6.279128e+07,24,16216.0


Merge in the weather

In [90]:
# Just cut off the 'United States'
weather['Name'] = weather['Name'].str[:-len(", United States")]
weather

,Name,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions
0,"Atlanta, GA",2018-01-01,28.2,18.7,22.3,5.4,NaN,0.00,0.00,0.00,14.6,328.25,25.3,9.9,70.0,51.69,Partially cloudy
1,"Atlanta, GA",2018-01-02,35.1,12.1,23.5,3.1,NaN,0.00,0.00,0.00,6.9,182.33,NaN,9.9,11.6,50.16,Clear
2,"Atlanta, GA",2018-01-03,37.6,26.4,30.7,19.8,NaN,0.00,0.00,0.00,12.0,206.58,18.3,9.9,45.5,36.36,Partially cloudy
3,"Atlanta, GA",2018-01-04,33.6,21.1,26.9,8.5,NaN,0.00,0.00,0.00,20.6,311.58,29.9,9.9,2.7,42.91,Clear
4,"Atlanta, GA",2018-01-05,36.8,16.5,25.6,3.6,NaN,0.00,0.00,0.00,14.2,306.17,23.0,9.9,10.9,40.79,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,"Washington, DC",2020-01-04,58.9,46.9,52.8,40.3,NaN,0.34,0.00,0.00,16.7,222.42,28.7,7.2,79.4,92.45,"Rain, Overcast"
7376,"Washington, DC",2020-01-05,46.0,35.4,40.6,29.5,NaN,0.00,0.00,0.00,21.5,299.08,36.6,9.9,72.3,54.88,Partially cloudy
7377,"Washington, DC",2020-01-06,55.0,35.3,44.1,31.3,NaN,0.00,0.00,0.00,16.9,226.13,27.7,9.9,46.2,49.96,Partially cloudy
7378,"Washington, DC",2020-01-07,45.9,32.9,36.8,25.8,NaN,0.23,0.74,0.74,13.7,167.50,18.3,8.6,66.4,72.52,"Snow, Partially cloudy"


In [91]:
# Only take the interesting columns
interestWeather = weather[['Name', 'Date time', 'Temperature', 'Precipitation', 'Snow', 'Wind Speed', 'Visibility']]
originMerged = pd.merge(
    modelData,
    interestWeather,
    how = 'left',
    left_on = ['origin_city_name', 'fl_date'],
    right_on = ['Name', 'Date time']
)

weatherMerged = pd.merge(
    originMerged,
    interestWeather,
    how = 'left',
    left_on = ['dest_city_name', 'fl_date'],
    right_on = ['Name', 'Date time'],
    suffixes = ('_origin', '_dest')
)
weatherMerged

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,Snow_origin,Wind Speed_origin,Visibility_origin,Name_dest,Date time_dest,Temperature_dest,Precipitation_dest,Snow_dest,Wind Speed_dest,Visibility_dest
0,2018-04-18,DL,DL,DL,1781,DL,N946DN,1781,10397,ATL,...,0.0,17.6,9.9,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,2019-04-16,AA,AA,AA,2828,AA,N991AN,2828,10397,ATL,...,0.0,8.0,9.9,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,2018-04-26,DL,DL,DL,1987,DL,N934DL,1987,13303,MIA,...,NaN,NaN,NaN,"Atlanta, GA",2018-04-26,55.3,0.20,0.0,9.4,9.0
3,2019-04-12,DL,DL,DL,1210,DL,N914DN,1210,13303,MIA,...,NaN,NaN,NaN,"Atlanta, GA",2019-04-12,71.0,0.06,0.0,14.4,9.6
4,2019-04-11,AA,AA,AA,2669,AA,N842NN,2669,13303,MIA,...,NaN,NaN,NaN,"New York, NY",2019-04-11,45.6,0.00,0.0,9.2,9.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316605,2019-01-14,B6,B6,B6,499,B6,N562JB,499,12953,LGA,...,0.0,16.1,9.9,NaN,NaT,NaN,NaN,NaN,NaN,NaN
316606,2019-11-06,NK,NK,NK,968,NK,N687NK,968,13930,ORD,...,0.0,19.5,9.3,NaN,NaT,NaN,NaN,NaN,NaN,NaN
316607,2019-02-04,B6,B6,B6,499,B6,N583JB,499,12953,LGA,...,0.0,9.4,9.5,NaN,NaT,NaN,NaN,NaN,NaN,NaN
316608,2019-02-17,B6,B6,B6,499,B6,N640JB,499,12953,LGA,...,0.0,17.0,9.8,NaN,NaT,NaN,NaN,NaN,NaN,NaN


Now drop columns which aren't needed

In [99]:
keepcols = [
    'crs_arr_time', 'distance', 'plane_speed', 'carrier_delay',
    'origin_ratio', 'dest_ratio', 'distance_ratio', 'month_ratio',
    'hour_ratio', 'Temperature_origin', 'Precipitation_origin',
    'Snow_origin', 'Wind Speed_origin', 'Visibility_origin',
    'Temperature_dest', 'Precipitation_dest', 'Snow_dest',
    'Wind Speed_dest', 'Visibility_dest', 'target'
]
modelData = weatherMerged[keepcols].copy()
modelData

,crs_arr_time,distance,plane_speed,carrier_delay,origin_ratio,dest_ratio,distance_ratio,month_ratio,hour_ratio,Temperature_origin,Precipitation_origin,Snow_origin,Wind Speed_origin,Visibility_origin,Temperature_dest,Precipitation_dest,Snow_dest,Wind Speed_dest,Visibility_dest,target
0,1342,594.0,0.147540,0.163906,5.761939e+05,176004.521277,23328.456293,4.474688e+07,8921628.0,67.7,0.00,0.0,17.6,9.9,NaN,NaN,NaN,NaN,NaN,-17.0
1,1340,594.0,0.143477,4.239740,5.761939e+05,176004.521277,23328.456293,4.474688e+07,8921628.0,60.1,0.00,0.0,8.0,9.9,NaN,NaN,NaN,NaN,NaN,9.0
2,1324,594.0,0.158601,0.163906,3.112660e+05,413880.585106,23328.456293,4.474688e+07,8921628.0,NaN,NaN,NaN,NaN,NaN,55.3,0.20,0.0,9.4,9.0,-20.0
3,1309,594.0,0.148119,0.163906,3.112660e+05,413880.585106,23328.456293,4.474688e+07,8921628.0,NaN,NaN,NaN,NaN,NaN,71.0,0.06,0.0,14.4,9.6,-4.0
4,1300,1096.0,0.143921,4.239740,3.112660e+05,968278.457447,4990.204588,4.474688e+07,8921628.0,NaN,NaN,NaN,NaN,NaN,45.6,0.00,0.0,9.2,9.9,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316605,2400,950.0,0.137919,14.760556,5.650231e+05,465330.319149,8279.603224,4.016998e+07,16216.0,26.9,0.00,0.0,16.1,9.9,NaN,NaN,NaN,NaN,NaN,55.0
316606,2400,621.0,0.136459,1.105945,2.238224e+06,130999.734043,6480.099194,3.189132e+07,16216.0,40.4,0.00,0.0,19.5,9.3,NaN,NaN,NaN,NaN,NaN,2.0
316607,2400,950.0,0.140308,9.964439,5.650231e+05,465330.319149,8279.603224,6.279128e+07,16216.0,47.7,0.00,0.0,9.4,9.5,NaN,NaN,NaN,NaN,NaN,-22.0
316608,2400,950.0,0.137931,9.964439,5.650231e+05,465330.319149,8279.603224,6.279128e+07,16216.0,31.9,0.03,0.0,17.0,9.8,NaN,NaN,NaN,NaN,NaN,59.0


Fill in any missing values using the average value in a column. Things like temperature and visibility don't default to 0, and columns that _do_ default to 0 (like precip) will have a significant amount of columns at 0 already to bring the average there.

In [100]:
temp_avg = modelData[['Temperature_origin', 'Temperature_dest']].median().mean() # Whatever, it's close enough
precip_avg = modelData[['Precipitation_origin', 'Precipitation_dest']].median().mean()
snow_avg = modelData[['Snow_origin', 'Snow_dest']].median().mean()
wind_avg = modelData[['Wind Speed_origin', 'Wind Speed_dest']].median().mean()
vis_avg = modelData[['Visibility_origin', 'Visibility_dest']].median().mean()

modelData[['Temperature_origin', 'Temperature_dest']] = \
    modelData[['Temperature_origin', 'Temperature_dest']].fillna(temp_avg)
modelData[['Precipitation_origin', 'Precipitation_dest']] = \
    modelData[['Precipitation_origin', 'Precipitation_dest']].fillna(precip_avg)
modelData[['Snow_origin', 'Snow_dest']] = \
    modelData[['Snow_origin', 'Snow_dest']].fillna(snow_avg)
modelData[['Wind Speed_origin', 'Wind Speed_dest']] = \
    modelData[['Wind Speed_origin', 'Wind Speed_dest']].fillna(wind_avg)
modelData[['Visibility_origin', 'Visibility_dest']] = \
    modelData[['Visibility_origin', 'Visibility_dest']].fillna(vis_avg)

modelData

,crs_arr_time,distance,plane_speed,carrier_delay,origin_ratio,dest_ratio,distance_ratio,month_ratio,hour_ratio,Temperature_origin,Precipitation_origin,Snow_origin,Wind Speed_origin,Visibility_origin,Temperature_dest,Precipitation_dest,Snow_dest,Wind Speed_dest,Visibility_dest,target
0,1342,594.0,0.147540,0.163906,5.761939e+05,176004.521277,23328.456293,4.474688e+07,8921628.0,67.7,0.00,0.0,17.60,9.9,61.6,0.00,0.0,12.25,9.8,-17.0
1,1340,594.0,0.143477,4.239740,5.761939e+05,176004.521277,23328.456293,4.474688e+07,8921628.0,60.1,0.00,0.0,8.00,9.9,61.6,0.00,0.0,12.25,9.8,9.0
2,1324,594.0,0.158601,0.163906,3.112660e+05,413880.585106,23328.456293,4.474688e+07,8921628.0,61.6,0.00,0.0,12.25,9.8,55.3,0.20,0.0,9.40,9.0,-20.0
3,1309,594.0,0.148119,0.163906,3.112660e+05,413880.585106,23328.456293,4.474688e+07,8921628.0,61.6,0.00,0.0,12.25,9.8,71.0,0.06,0.0,14.40,9.6,-4.0
4,1300,1096.0,0.143921,4.239740,3.112660e+05,968278.457447,4990.204588,4.474688e+07,8921628.0,61.6,0.00,0.0,12.25,9.8,45.6,0.00,0.0,9.20,9.9,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316605,2400,950.0,0.137919,14.760556,5.650231e+05,465330.319149,8279.603224,4.016998e+07,16216.0,26.9,0.00,0.0,16.10,9.9,61.6,0.00,0.0,12.25,9.8,55.0
316606,2400,621.0,0.136459,1.105945,2.238224e+06,130999.734043,6480.099194,3.189132e+07,16216.0,40.4,0.00,0.0,19.50,9.3,61.6,0.00,0.0,12.25,9.8,2.0
316607,2400,950.0,0.140308,9.964439,5.650231e+05,465330.319149,8279.603224,6.279128e+07,16216.0,47.7,0.00,0.0,9.40,9.5,61.6,0.00,0.0,12.25,9.8,-22.0
316608,2400,950.0,0.137931,9.964439,5.650231e+05,465330.319149,8279.603224,6.279128e+07,16216.0,31.9,0.03,0.0,17.00,9.8,61.6,0.00,0.0,12.25,9.8,59.0


# Predictions

In [144]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

y = modelData.target
X = modelData.drop(columns='target')

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

lr = LinearRegression()
lr.fit(Xtrain, ytrain)
lr.score(Xtest, ytest)

0.03938103158679851

In [145]:
import statsmodels.api as sm

Xconst = sm.add_constant(Xtrain)
testConst = sm.add_constant(Xtest)

model = sm.OLS(ytrain, Xconst)
res = model.fit()

res.summary()

E:\Users\eboyc\anaconda3\envs\bootcamp\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     578.4
Date:                Thu, 23 Sep 2021   Prob (F-statistic):               0.00
Time:                        14:56:46   Log-Likelihood:            -1.2654e+06
No. Observations:              237457   AIC:                         2.531e+06
Df Residuals:                  237437   BIC:                         2.531e+06
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    8.4843      2.221      3.820      0.000       4.132      12.837
crs_arr_time            -0.0007      0.000     -1.571      0.116      -0.001       0.000
distance                -0.0017      0.000     -7.556      0.000      -0.002      -0.001
plane_speed             12.4349      6.757      1.840      0.066      -0.808      25.678
carrier_delay            0.7517      0.026     29.232      0.000       0.701       0.802
origin_ratio          2.199e-06   1.91e-07     11.539      0.000    1.83e-06    2.57e-06
dest_ratio            2.398e-06   2.23e-07     10.771      0.000    1.96e-06    2.83e-06
distance_ratio        8.369e-05   1.32e-05      6.356      0.000    5.79e-05       0.000
month_ratio            2.59e-08   4.22e-09      6.132      0.000    1.76e-08    3.42e-08
hour_ratio            2.956e-07   1.25e-08     23.608      0.000    2.71e-07     3.2e-07
Temperature_origin       0.0444      0.010      4.600      0.000       0.026       0.063
Precipitation_origin    15.4924      0.535     28.953      0.000      14.444      16.541
Snow_origin             11.1017      0.553     20.074      0.000      10.018      12.186
Wind Speed_origin        0.3669      0.032     11.376      0.000       0.304       0.430
Visibility_origin       -1.8421      0.123    -14.998      0.000      -2.083      -1.601
Temperature_dest         0.0519      0.010      5.365      0.000       0.033       0.071
Precipitation_dest      16.3767      0.543     30.162      0.000      15.312      17.441
Snow_dest               12.3153      0.549     22.434      0.000      11.239      13.391
Wind Speed_dest          0.4505      0.032     13.963      0.000       0.387       0.514
Visibility_dest         -1.8951      0.123    -15.457      0.000      -2.135      -1.655
==============================================================================
Omnibus:                   344878.909   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        203405322.838
Skew:                           8.634   Prob(JB):                         0.00
Kurtosis:                     145.338   Cond. No.                     4.72e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.72e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [146]:
import xgboost as xgb
from sklearn.metrics import r2_score

xgreg = xgb.XGBRegressor(
    max_depth = 5,
    n_estimators = 50
)
xgreg.fit(Xtrain, ytrain)
pred = xgreg.predict(Xtest)

r2_score(ytest, pred)

0.0704680749841442